In [2]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
torch = { version = "=1.12.1+cu113", source = "torch" }

[[tool.poetry.source]]
name = "torch"
url = "https://download.pytorch.org/whl/cu113"
secondary = true
default = false


In [3]:
# df = pd.read_parquet('C:/Users/sgopalakrish/Downloads/intellizenz-model-training/data/export_features_2016_2020_v3.parquet.gzip')
df = pd.read_parquet('/Users/saravana/Documents/Work/Master Thesis/reclamation-processing/data/export_anonymized_features_2016_2020.parquet.gzip')

pd.set_option('display.max_columns', None)
display(df.head())

,anonymized_band,anonymized_promoter,anonymized_venue,tarif_bez,vg_state,vg_datum_season,vg_datum_month,vg_datum_day_of_week,vg_datum_year,vg_datum_von,veranst_segment,vg_inkasso
ID,,,,,,,,,,,,
9938766,Klapp Dehmel AG & Co. OHG,Wiek Löchel AG,turnhalle,U-K (MUSIKER),BAVARIA,0,1,5,0,2014-02-08,2,210.00
9939272,Karz Ritter KGaA,Peukert Neuschäfer GmbH,grundschule,U-V II. 1 (+II 2),THURINGIA,0,1,5,0,2014-02-22,1,85.06
9939279,Klemm Ullrich OHG mbH,Naser Klemt KG,werk,U-V II. 1 (+II 2),BAVARIA,0,1,1,0,2014-02-18,1,90.20
9939280,Albers Klemt GmbH & Co. OHG,Patberg AG & Co. KG,kulturhaus,U-V II. 1 (+II 2),BAVARIA,1,2,5,0,2014-03-29,2,206.12
9939281,Albers Klemt GmbH & Co. OHG,Patberg AG & Co. KG,kulturhaus,U-V II. 1 (+II 2),BAVARIA,1,2,4,0,2014-03-21,2,206.12


In [4]:
df.shape

(1742197, 12)

In [5]:
df['tarif_bez'].unique()

['U-K (MUSIKER)', 'U-V II. 1 (+II 2)', 'U-K (MUSIKER) MINDESTVERGUETUNG', 'U-K I (+ II 2B) - MIT SONSTIGEM GELDWERTEN VO..., 'U-K II. (MUSIKER) BIS 50 MIN', ..., 'WR-HZ (MUSIKER)', 'KONZERT CHORVERBAENDE OHNE EINTRITTSGELD (7,5%)', 'Z I. 1./3. (MUSIKER)', 'BM I. 1. / 2.', 'RV/L NR. 1 (3) I 1. PROZENTABRECHNUNG TONTRAE...]
Length: 87
Categories (91, object): ['BM I. 1. / 2.', 'BM I.1. BIS 25 MINUTEN', 'CHORVERBAND KONZERT E-MUSIK MIT EINTRITTSGELD', 'CHORVERBAND KONZERT E-MUSIK OHNE EINTRITTSGELD', ..., 'WR-KS I. 2 EINZEL (MUSIKER)', 'WR-NWSP I.', 'WR-VR-B 1 I (MUSIKER)', 'Z I. 1./3. (MUSIKER)']

### Get all unique values of Tarif that contains 'U-ST'

In [6]:
tarif_u_st_df = df.loc[df['tarif_bez'].str.contains('U-ST ')].copy()

In [7]:
display(tarif_u_st_df.head())

,anonymized_band,anonymized_promoter,anonymized_venue,tarif_bez,vg_state,vg_datum_season,vg_datum_month,vg_datum_day_of_week,vg_datum_year,vg_datum_von,veranst_segment,vg_inkasso
ID,,,,,,,,,,,,
9943647,Lübs Textor GmbH & Co. KG,Buchholz Langern AG,Juliane-Wiek-Gasse,U-ST (MUSIKER),SAXONY-ANHALT,0,0,5,0,2014-01-18,1,55.44
9946796,Hein Dörr AG,Kambs e.G.,Dogan-Ziegert-Allee,U-ST (MUSIKER),BAVARIA,1,3,3,0,2014-04-24,2,163.20
9949751,Geisler Reinhardt AG,Hesse Keudel GbR,Theodora-Kobelt-Platz,U-ST (MUSIKER),THURINGIA,1,3,5,0,2014-04-26,2,106.20
9949924,Ehlert Briemer GmbH & Co. KG,Gotthard,Senol-Bachmann-Straße,U-ST (MUSIKER),RHINELAND-PALATINATE,1,3,6,0,2014-04-20,1,84.96
9950662,Pohl Rogge AG & Co. KG,Roht GmbH,Klaus-D.-Jopich-Gasse,U-ST (MUSIKER),BRANDENBURG,1,4,6,0,2014-05-04,1,69.30


## Get the tariffs specific and not-specific for each class 

In [17]:
class_zero_tariffs = []
class_one_tariffs = []
class_two_tariffs = []

never_class_zero = []
never_class_one = []
never_class_two = []

for each in df['tarif_bez'].unique():
    classes = df[df['tarif_bez'] == each]["veranst_segment"].values.tolist()
    class_count = len(set(classes))
    if class_count == 2:
        not_occuring_class = list(set([0,1,2]).difference(set(classes)))[0]
        if not_occuring_class == 0:
            never_class_zero.append(each)
        elif not_occuring_class == 1:
            never_class_one.append(each)
        else:
            never_class_two.append(each)

    if class_count == 1:
        each_class = classes[0]
        if each_class == 0:
            class_zero_tariffs.append(each)
        elif each_class == 1:
            class_one_tariffs.append(each)
        else:
            class_two_tariffs.append(each)

print(never_class_zero)      
print(len(never_class_zero))
print(never_class_one)
print(len(never_class_one))
print(never_class_two)
print(len(never_class_two))

['U-ST (MUSIKER)', 'M-SP II. 1+2 (MUSIKER)']
2
['U-VK I IM FREIEN NACH QM', 'KONZERT CHORVERBAENDE OHNE EINTRITTSGELD (5%)']
2
['VK I 1 VARIETE- UND KABARETTBETRIEBE KONSUMATION (MUSIKER) NEU', 'E-P', 'VARIETE I MINDESTVERGUETUNG (MUSIKER)', 'E (TONTRAEGER)', 'KONZERT CHORVERBAENDE OHNE EINTRITTSGELD (10%)', 'KONZERT CHORVERBAENDE OHNE EINTRITTSGELD (7,5%)', 'BM I. 1. / 2.']
7


In [ ]:
ten_class_zero_tariffs = ['WR-NWSP I.', 'U-VK II 3', 'PV/ST NR.3(1) KURSE (MUSIK)', 
 'U-VK I (+III 2A) - FUER MOEGLICHEN ZEITZUSCHLAG SCHNELLERFASSUNG', 
 'U-VK II 2', 'CHORVERBAND KONZERT E-MUSIK OHNE EINTRITTSGELD', 
 'WR-BEST (MUSIKER)', 'PV/ST NR. 3(1) ZIFF. 3', 
 'WR-KS I. 2 EINZEL (MUSIKER)', 'WR-HZ (MUSIKER)']

two_class_two_tariffs = ['U-ST I (MUSIKER) NL', 'RV/L NR. 1 (3) I 1. PROZENTABRECHNUNG TONTRAEGER']

never_class_zero = ['U-ST (MUSIKER)', 'M-SP II. 1+2 (MUSIKER)']

never_class_one = ['U-VK I IM FREIEN NACH QM', 'KONZERT CHORVERBAENDE OHNE EINTRITTSGELD (5%)']

never_class_two = ['VK I 1 VARIETE- UND KABARETTBETRIEBE KONSUMATION (MUSIKER) NEU', 
                   'E-P', 'VARIETE I MINDESTVERGUETUNG (MUSIKER)', 'E (TONTRAEGER)', 
                   'KONZERT CHORVERBAENDE OHNE EINTRITTSGELD (10%)', 
                   'KONZERT CHORVERBAENDE OHNE EINTRITTSGELD (7,5%)', 
                   'BM I. 1. / 2.']

In [ ]:
values_u_st = []
for each in tarif_u_st_df['tarif_bez'].values:
    values_u_st.append(each)

In [ ]:
new_tarif_u_st_df = pd.DataFrame({'tarif_bez':values_u_st})

In [ ]:
fig, ax = plt.subplots()
new_tarif_u_st_df['tarif_bez'].value_counts().plot(ax=ax, kind='bar', xlabel='Tarifs', ylabel='Frequency')
plt.show()
# U-ST I. (MUSIKER) - 106539
# U-ST I (MUSIKER) NL - 27919
# U-ST (MUSIKER) - 3928

In [ ]:
tarif_u_st_df.shape

### vg_state

In [ ]:
new_tarif_u_st_df['state'] = tarif_u_st_df['vg_state'].values

In [ ]:
new_tarif_u_st_df['vg_raum_keywords'] = tarif_u_st_df['vg_raum_keywords'].values

In [ ]:
new_tarif_u_st_df['inkasso'] = tarif_u_st_df['vg_inkasso'].values

In [ ]:
new_tarif_u_st_df['veranst_segment'] = tarif_u_st_df['veranst_segment'].values

In [ ]:
new_tarif_u_st_df['venue'] = [','.join(map(str, l)) for l in new_tarif_u_st_df['vg_raum_keywords']]

In [ ]:
u_st_nl_df = df.loc[df['tarif_bez']=='U-ST I (MUSIKER) NL']
u_st_nl_df.shape

In [ ]:
display(new_tarif_u_st_df.head())

In [ ]:
# u_st_df = new_tarif_u_st_df.loc[new_tarif_u_st_df['tarif_bez']=='U-ST (MUSIKER)'] # veranst segment ==> 1 to 2 - {>50€}
u_st_i_df = new_tarif_u_st_df.loc[new_tarif_u_st_df['tarif_bez']=='U-ST I. (MUSIKER)'] # veranst segment ==> 0 to 2 - {>0€}
# u_st_nl_df = new_tarif_u_st_df.loc[new_tarif_u_st_df['tarif_bez']=='U-ST I (MUSIKER) NL'] # veranst segment ==> 2 to 2 {>100€}

state = ['bavaria', 'thuringia', 'baden-wuerttemberg', 'rhineland-palatinate', 
         'brandenburg', 'berlin', 'lower saxony', 'schleswig-holstein', 'hamburg', 
         'hesse', 'saarland', 'north rhine-westphalia', 'saxony-anhalt', 
         'saxony', 'mecklenburg-western pomerania', 'bremen']

for each in state:
    st = each.upper()
    print('--------State---------: ',st)
    specific_state_df = u_st_i_df.loc[u_st_i_df['state']==st]
    # print('Venues available here are: ',specific_state_df.venue.value_counts())
    # print(specific_state_df.venue.value_counts())
    print('Total number of venues: ',len(specific_state_df.venue.values))

    if st == 'SAXONY':
        kulturzentrum_df = specific_state_df.loc[specific_state_df.venue.str.contains('kulturzentrum')==True]
        print('The collection in kulturzentrum is :', kulturzentrum_df.inkasso)
        print('The kulturzentrum min collection is: {}, max is:{}'.format(kulturzentrum_df.inkasso.min(), kulturzentrum_df.inkasso.max()))
        print('The venue is: ', kulturzentrum_df.venue)

        kirche_df = specific_state_df.loc[specific_state_df.venue.str.contains('kirche')==True]
        # print('The collection in kulturzentrum is :', kirche_df.inkasso)
        print('The kirche min collection is: {}, max is:{}'.format(kirche_df.inkasso.min(), kirche_df.inkasso.max()))
        # print('The venue is: ', kirche_df.venue)

        hotel_df = specific_state_df.loc[specific_state_df.venue.str.contains('hotel')==True]
        # print('The collection in hotel is :', hotel_df.inkasso)
        print('The hotel min collection is: {}, max is:{}'.format(hotel_df.inkasso.min(), hotel_df.inkasso.max()))

        cafe_df = specific_state_df.loc[specific_state_df.venue.str.contains('cafe')==True]
        # print('The collection in cafe is :', cafe_df.inkasso)
        print('The cafe min collection is: {}, max is:{}'.format(cafe_df.inkasso.min(), cafe_df.inkasso.max()))

        theater_df = specific_state_df.loc[specific_state_df.venue.str.contains('theater')==True]
        # print('The collection in theater is :', theater_df.inkasso)
        print('The theater min collection is: {}, max is:{}'.format(theater_df.inkasso.min(), theater_df.inkasso.max()))

        club_df = specific_state_df.loc[specific_state_df.venue.str.contains('club')==True]
        # print('The collection in club is :', club_df.inkasso)
        print('The club min collection is: {}, max is:{}'.format(club_df.inkasso.min(), club_df.inkasso.max()))

        halle_df = specific_state_df.loc[specific_state_df.venue.str.contains('halle')==True]
        # print('The collection in halle is :', halle_df.inkasso)
        print('The halle min collection is: {}, max is:{}'.format(halle_df.inkasso.min(), halle_df.inkasso.max()))

        gaststaette_df = specific_state_df.loc[specific_state_df.venue.str.contains('gaststaette')==True]
        # print('The collection in gaststaette is :', gaststaette_df.inkasso)
        print('The gaststaette min collection is: {}, max is:{}'.format(gaststaette_df.inkasso.min(), gaststaette_df.inkasso.max()))

        festhalle_df = specific_state_df.loc[specific_state_df.venue.str.contains('festhalle')==True]
        # print('The collection in festhalle is :', festhalle_df.inkasso)
        print('The festhalle min collection is: {}, max is:{}'.format(festhalle_df.inkasso.min(), festhalle_df.inkasso.max()))

        festzelt_df = specific_state_df.loc[specific_state_df.venue.str.contains('festzelt')==True]
        # print('The collection in festzelt is :', festzelt_df.inkasso)
        print('The festzelt min collection is: {}, max is:{}'.format(festzelt_df.inkasso.min(), festzelt_df.inkasso.max()))

        schloss_df = specific_state_df.loc[specific_state_df.venue.str.contains('schloss')==True]
        # print('The collection in schloss is :', schloss_df.inkasso)
        print('The schloss min collection is: {}, max is:{}'.format(schloss_df.inkasso.min(), schloss_df.inkasso.max()))

        pub_df = specific_state_df.loc[specific_state_df.venue.str.contains('pub')==True]
        # print('The collection in pub is :', pub_df.inkasso)
        print('The pub min collection is: {}, max is:{}'.format(pub_df.inkasso.min(), pub_df.inkasso.max()))

        stadthalle_df = specific_state_df.loc[specific_state_df.venue.str.contains('stadthalle')==True]
        # print('The collection in stadthalle is :', stadthalle_df.inkasso)
        print('The stadthalle min collection is: {}, max is:{}'.format(stadthalle_df.inkasso.min(), stadthalle_df.inkasso.max()))

        park_df = specific_state_df.loc[specific_state_df.venue.str.contains('park')==True]
        # print('The collection in park is :', park_df.inkasso)
        print('The park min collection is: {}, max is:{}'.format(park_df.inkasso.min(), park_df.inkasso.max()))

        gasthof_df = specific_state_df.loc[specific_state_df.venue.str.contains('gasthof')==True]
        # print('The collection in gasthof is :', gasthof_df.inkasso)
        print('The gasthof min collection is: {}, max is:{}'.format(gasthof_df.inkasso.min(), gasthof_df.inkasso.max()))

        kabarett_df = specific_state_df.loc[specific_state_df.venue.str.contains('kabarett')==True]
        # print('The collection in kabarett is :', kabarett_df.inkasso)
        print('The kabarett min collection is: {}, max is:{}'.format(kabarett_df.inkasso.min(), kabarett_df.inkasso.max()))

        arena_df = specific_state_df.loc[specific_state_df.venue.str.contains('arena')==True]
        # print('The collection in arena is :', arena_df.inkasso)
        print('The arena min collection is: {}, max is:{}'.format(arena_df.inkasso.min(), arena_df.inkasso.max()))

        schlachthof_df = specific_state_df.loc[specific_state_df.venue.str.contains('schlachthof')==True]
        # print('The collection in schlachthof is :', schlachthof_df.inkasso)
        print('The schlachthof min collection is: {}, max is:{}'.format(schlachthof_df.inkasso.min(), schlachthof_df.inkasso.max()))

        wandelhalle_df = specific_state_df.loc[specific_state_df.venue.str.contains('wandelhalle')==True]
        # print('The collection in wandelhalle is :', wandelhalle_df.inkasso)
        print('The wandelhalle min collection is: {}, max is:{}'.format(wandelhalle_df.inkasso.min(), wandelhalle_df.inkasso.max()))

        turnhalle_df = specific_state_df.loc[specific_state_df.venue.str.contains('turnhalle')==True]
        # print('The collection in turnhalle is :', turnhalle_df.inkasso)
        print('The turnhalle min collection is: {}, max is:{}'.format(turnhalle_df.inkasso.min(), turnhalle_df.inkasso.max()))

        buergerhaus_df = specific_state_df.loc[specific_state_df.venue.str.contains('buergerhaus')==True]
        # print('The collection in buergerhaus is :', buergerhaus_df.inkasso)
        print('The buergerhaus min collection is: {}, max is:{}'.format(buergerhaus_df.inkasso.min(), buergerhaus_df.inkasso.max()))

        museum_df = specific_state_df.loc[specific_state_df.venue.str.contains('museum')==True]
        # print('The collection in museum is :', museum_df.inkasso)
        print('The museum min collection is: {}, max is:{}'.format(museum_df.inkasso.min(), museum_df.inkasso.max()))

        rathaus_df = specific_state_df.loc[specific_state_df.venue.str.contains('rathaus')==True]
        # print('The collection in rathaus is :', rathaus_df.inkasso)
        print('The rathaus min collection is: {}, max is:{}'.format(rathaus_df.inkasso.min(), rathaus_df.inkasso.max()))

        staatsbad_df = specific_state_df.loc[specific_state_df.venue.str.contains('staatsbad')==True]
        # print('The collection in staatsbad is :', staatsbad_df.inkasso)
        print('The staatsbad min collection is: {}, max is:{}'.format(staatsbad_df.inkasso.min(), staatsbad_df.inkasso.max()))

        zelt_df = specific_state_df.loc[specific_state_df.venue.str.contains('zelt')==True]
        # print('The collection in zelt is :', zelt_df.inkasso)
        print('The zelt min collection is: {}, max is:{}'.format(zelt_df.inkasso.min(), zelt_df.inkasso.max()))

        jazz_df = specific_state_df.loc[specific_state_df.venue.str.contains('jazz')==True]
        # print('The collection in jazz is :', jazz_df.inkasso)
        print('The jazz min collection is: {}, max is:{}'.format(jazz_df.inkasso.min(), jazz_df.inkasso.max()))

        forum_df = specific_state_df.loc[specific_state_df.venue.str.contains('forum')==True]
        # print('The collection in forum is :', forum_df.inkasso)
        print('The forum min collection is: {}, max is:{}'.format(forum_df.inkasso.min(), forum_df.inkasso.max()))

        gymnasium_df = specific_state_df.loc[specific_state_df.venue.str.contains('gymnasium')==True]
        # print('The collection in gymnasium is :', gymnasium_df.inkasso)
        print('The gymnasium min collection is: {}, max is:{}'.format(gymnasium_df.inkasso.min(), gymnasium_df.inkasso.max()))

        schule_df = specific_state_df.loc[specific_state_df.venue.str.contains('schule')==True]
        # print('The collection in schule is :', schule_df.inkasso)
        print('The schule min collection is: {}, max is:{}'.format(schule_df.inkasso.min(), schule_df.inkasso.max()))

        sporthalle_df = specific_state_df.loc[specific_state_df.venue.str.contains('sporthalle')==True]
        # print('The collection in sporthalle is :', sporthalle_df.inkasso)
        print('The sporthalle min collection is: {}, max is:{}'.format(sporthalle_df.inkasso.min(), sporthalle_df.inkasso.max()))

    # # indices = specific_state_df.index[specific_state_df.venue.str.contains('kulturzentrum').any()==True].tolist()
    # print('The venue kulturzentrum is present of following indices: ',specific_state_df.venue.str.contains('kulturzentrum').sum())
    
    # if specific_state_df.venue.str.contains('kulturzentrum').any():
    #     print('Contains kulturzentrum--------------------------------------------------------------')
    # zero_to_fifty_df = specific_state_df.loc[specific_state_df['veranst_segment']==0]
    # print('The venues where inkasso is less than 50€ :',zero_to_fifty_df.vg_raum_keywords)
    # print('The max inkasso: ', specific_state_df.inkasso.max())
    # print('The min inkasso: ', specific_state_df.inkasso.min())
    # print('The max segment: ', specific_state_df.veranst_segment.max())
    # print('The min segment: ', specific_state_df.veranst_segment.min())
    # print('------------------------')

In [ ]:
# VENUES 
# place_kirche	place_hotel	place_cafe	
# place_theater	place_club	place_halle	
# place_gaststaette	place_festhalle	place_kulturzentrum	
# place_festzelt	place_schloss	place_pub	
# place_stadthalle	place_park	place_gasthof	
# place_kabarett	place_arena	place_schlachthof	
# place_wandelhalle	place_turnhalle	place_buergerhaus	
# place_museum	place_rathaus	place_staatsbad	
# place_zelt	place_jazz	place_forum	
# place_gymnasium	place_schule	place_sporthalle

In [ ]:
venues = ['kirche', 'hotel', 'cafe',
'theater', 'club', 'halle',	
'gaststaette', 'festhalle', 'kulturzentrum', 	
'festzelt', 'schloss', 'pub', 	
'stadthalle', 'park', 'gasthof', 
'kabarett', 'arena', 'schlachthof', 
'wandelhalle', 'turnhalle', 'buergerhaus', 
'museum', 'rathaus', 'staatsbad', 
'zelt', 'jazz', 'forum', 
'gymnasium', 'schule', 'sporthalle']

In [ ]:
# tarif_u_st_df.plot(x='tarif_bez', y=['state_bavaria'], kind='line', figsize=(10,10))
chart = sns.relplot(data=tarif_u_st_df, x='tarif_bez', y='vg_state', style='place_kirche', kind='line')

In [ ]:
flat_list_vg_raum_keywords = [item for sublist in tarif_u_st_df['vg_raum_keywords'] for item in sublist]

In [ ]:
tarif_u_st_in_one_state_df = tarif_u_st_df.loc[tarif_u_st_df['vg_state']=='SAXONY']

In [ ]:
display(tarif_u_st_in_one_state_df.head())

In [ ]:
chart = sns.countplot(data=tarif_u_st_df, x='vg_raum_wo_stopwords', hue='vg_state')

### Get all unique values of Tarif that contains 'U-V'

In [ ]:
tarif_u_v_df = df.loc[df['tarif_bez'].str.contains('U-V ')].copy()

In [ ]:
tarif_u_v_df.shape

In [ ]:
uv_values = []
for each in tarif_u_v_df['tarif_bez'].values:
    uv_values.append(each)

In [ ]:
new_tarif_u_v_df = pd.DataFrame({'tarif_bez':uv_values})

In [ ]:
fig, ax = plt.subplots()
new_tarif_u_v_df['tarif_bez'].value_counts().plot(ax=ax, kind='bar', xlabel='Tarifs', ylabel='Frequency')
plt.show()